## Imports

In [3]:
import pandas as pd
import numpy as np
import random as rd

In [148]:
%autosave 180

Autosaving every 180 seconds


## Generate the dataset

In [95]:
df = pd.DataFrame(np.random.randint(0, 2, size=(30, 8)), columns=['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8'])
df['y'] = df.sum(axis=1)
df

,x1,x2,x3,x4,x5,x6,x7,x8,y
0,1,1,0,0,0,0,0,0,2
1,0,0,1,1,0,0,0,0,2
2,0,1,0,1,1,0,1,0,4
3,1,1,0,0,1,0,1,0,4
4,1,0,1,1,1,0,0,1,5
5,0,0,1,1,0,0,1,0,3
6,1,0,0,1,0,0,0,1,3
7,1,0,0,0,0,0,0,0,1
8,1,0,1,0,1,0,1,1,5
9,1,1,0,0,0,0,0,1,3


In [96]:
df.y.sum()

123

## Adder with the Elman Recurrent Neural Network

We'll build an Elman Recurrent Neural Network to predict the sum of 8 bits. We have one neuron that has two inputs : the input bit $x_{n}$ and the previous output $f_{t-1}$.

As we want to have a linear model, the activation function will be the identity, and we don't need a bias.

In [145]:
class ElmanRNNAdder():
    
    def __init__(self):
        # Our two weights initialized randomly between -1 and 1
        self.Vx = rd.random() * 2 - 1
        self.Vf = rd.random() * 2 - 1
        
        # Learning rates
        self.lr_x = 0.001
        self.lr_f = 0.001
        
        # Number of features
        self.T = 0
        
        # Neuron output matrix
        self.f = None
        
        # Error
        self.err = []
    
    def forward(self, X):
        self.f = np.zeros((X.shape[0], X.shape[1] + 1))
        
        for t in range(self.T):
            self.f[:, t + 1] = self.Vf * self.f[:, t] + self.Vx * X[:, t]
        
        return self.f[:, -1]
    
    def compute_sse(self, y_hat, y):
        sse = 0.5 * np.sum(np.square(y_hat - y))
        
        return sse
    
    def backward(self, X, y, y_hat):
        dVx = 0
        dVf = 0
        
        for t in range(self.T):
            dVx += (np.sum((y_hat - y) * X[:, t]) * (self.Vf ** (self.T - t)))
            dVf += (np.sum((y_hat - y) * self.f[:, t - 1]) * (self.Vf ** (self.T - t)))
            
        print(dVx, dVf)
            
        self.Vx -= (self.lr_x * dVx)
        self.Vf -= (self.lr_f * dVf)
    
    def fit(self, X, y, nb_epoch=20):
        self.T = X.shape[1]
        
        for k in range(nb_epoch):
            y_hat = self.forward(X)

            self.err.append(self.compute_sse(y_hat, y))

            self.backward(X, y, y_hat)
        

model = ElmanRNNAdder()

In [146]:
model.fit(df.drop(columns='y').values, df['y'].values)

-509.4432609097483 352.97234927434516
0.2914910196063454 0.5851508572140703
-94.93885744283197 -51.680567612001624
0.38642987704917736 0.636831424826072
-109.2719877722229 -86.59836781886045
0.4957018648214003 0.7234297926449325
-138.24703744385968 -162.05469814634108
0.63394890226526 0.8854844907912736
-193.5325507368703 -366.82560542761286
0.8274814530021303 1.2523100962188864
1939.3103604873768 11740.531387166264
-1.1118289074852465 -10.488221290947378
3.3438168700874736e+16 5.1903104829094497e+23
-33438168700875.85 -5.19031048290945e+20
inf inf
-inf -inf
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
[392.9299003900713, 233.56627727347603, 211.51618396444567, 175.04741500879737, 90.65094194171506, 345.88744841288064, 1772345958582294.2, inf, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]


C:\Users\maxim\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: overflow encountered in square
C:\Users\maxim\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: overflow encountered in double_scalars
C:\Users\maxim\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: RuntimeWarning: overflow encountered in multiply
C:\Users\maxim\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in multiply
